# intro to llms

In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
print("is mpps available?", torch.backends.mps.is_available())
print(device)
# 超参数
block_size = 64  # 块大小用于 整数的长度
batch_size = 128 # 批处理用于并行运行的数
max_iters = 3000
learning_rate = 3e-3 # 一些常用的学习率 3e-4, 1e-3,1e-4
eval_iters = 100
eval_interval = 500
n_embd = 384
n_layer = 8
n_head = 8
dropout = 0.2

is mpps available? True
mps


# 数据处理,数据清理

In [53]:
chars = ""
with open("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
    chars = sorted(list(set(text)))
# 读取文本中的字符
vocab_size = len(chars)
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


# tokenizer 分词器  
由编码器和解码器组成
编码器将上面的数组每个元素转化为整数,解码器与之相反     

In [54]:
string_to_int = { ch: i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# print(encode('nice to meet you'))
# print(decode([67, 62, 56, 58, 1, 73, 68, 1, 66, 58, 58, 73, 1, 78, 68, 74]))
#['n', 'i', 'c', 'e', ' ', 't', 'o', ' ', 'm', 'e', 'e', 't', ' ', 'y', 'o', 'u']
data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


这里使用的是字符级的分词器,因此它可能会有40,000个标记,如果使用单词级的分词器,可能会有多国语言,那么可能会有几百万,几十亿甚至,万亿级的标记

# 训练集与验证集
使用80%的文本集进行训练,然后用20%的文本集进行验证,主要原因是,如果使用的100%的文本集全部用于去训练,那么模型只是单纯地记住了全部的文本,并不能让我们去验证训练的效果

In [55]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


x, y = get_batch("train")
print("inputs: \n", x)
print("targets: \n", y)

inputs: 
 tensor([[58, 78,  1,  ...,  1, 56, 68],
        [ 1,  1,  1,  ..., 28,  1, 44],
        [61,  1, 68,  ..., 65, 62, 59],
        ...,
        [ 1, 55, 58,  ..., 57, 62, 72],
        [68, 71, 72,  ..., 71, 73, 66],
        [58, 67,  9,  ..., 44, 61, 58]], device='mps:0')
targets: 
 tensor([[78,  1, 54,  ..., 56, 68, 66],
        [ 1,  1,  1,  ...,  1, 44, 39],
        [ 1, 68, 59,  ..., 62, 59, 73],
        ...,
        [55, 58, 54,  ..., 62, 72, 56],
        [71, 72, 58,  ..., 73, 66, 58],
        [67,  9,  1,  ..., 61, 58, 71]], device='mps:0')


# Head

In [64]:
class Head(nn.Module):
    """实现 MultiHead Attention 中的 Self-Attention"""
    def __init__(self, head_size) -> None:
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query =nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C, = x.shape
        k = self.key(x)
        q = self.query(x) # (B ,T , hs)
        # 计算注意力分数
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 #(B, T, hs) @ (B ,hs ,T) => (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x) # ( B, T,hs) 
        out = wei @ v # (B,T,T) @ (B,T,hs) => (B,T,hs)
        return out

# MultiHeadAttention

In [57]:
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, num_heads, head_size) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

# Feed Foward

In [58]:
class FeedFoward(nn.Module):
    """前馈网络"""
    def __init__(self, n_emdb) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout), # 防止过拟合,会丢掉 dropout 参数定义的神经元
        )
    
    def forward(self,x):
        return self.net(x)

# BLock

In [59]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head) -> None:
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)  # self Attation
        self.ffwd = FeedFoward(n_embd)  # feed foward
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

# GPT 类


In [60]:
class GPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, target=None):
        # logits = self.token_embedding_table(index) # 比如字符 “z” 可以映射成向量 [-1.05, 0.24, -0.46]
        B, T = index.shape
        tok_emv = self.token_embedding_table(index) # (B, T ,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emv + pos_emb # (B, T ,C)
        x = self.blocks(x)# (B, T ,C)
        x = self.ln_f(x)# (B, T ,C)
        logits = self.lm_head(x) # (B, T ,vocal_size)
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)  #
            target = target.view(B * T)
            loss = F.cross_entropy(
                logits, target
            )  # 它的作用是衡量模型的预测值和真实标签之间的差异。
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)  # (B, T +1)
        return index


model = GPT(vocab_size).to(device=device)

In [61]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [62]:
import pickle


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step:{iter},train loss: {losses['train']:.4f},val loss : {losses['val']:.4f}")
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    
    optimizer.zero_grad(set_to_none=True)
    # 反向传播
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

step:0,train loss: 4.4590,val loss : 4.4581
step:100,train loss: 3.1478,val loss : 3.1583
step:200,train loss: 3.1461,val loss : 3.1572
3.1622281074523926


In [63]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

RuntimeError: The expanded size of the tensor (65) must match the existing size (64) at non-singleton dimension 2.  Target sizes: [1, 65, 65].  Tensor sizes: [64, 64]